In [13]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [17]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [18]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [19]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
_________________________________________________________________


In [21]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
3/3 [==============================] - 0s 8ms/step - loss: 2.3005 - accuracy: 0.0833
Epoch 2/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2983 - accuracy: 0.1458
Epoch 3/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2965 - accuracy: 0.2396
Epoch 4/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2942 - accuracy: 0.2292
Epoch 5/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2922 - accuracy: 0.2396
Epoch 6/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2899 - accuracy: 0.2396
Epoch 7/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2879 - accuracy: 0.2604
Epoch 8/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2856 - accuracy: 0.3125
Epoch 9/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2833 - accuracy: 0.3229
Epoch 10/500
3/3 [==============================] - 0s 4ms/step - loss: 2.2806 - accuracy: 0.3125
Epoch 11/500
3/3 [===========

In [22]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [42]:
def webscraper(x):
  from urllib.request import Request, urlopen
  from bs4 import BeautifulSoup
  import requests
  from random import randint

  #x = inp

  root = "https://www.google.com"
  link = "https://www.google.com/search?q=top+10+{}+movies&oq=top+&aqs=chrome.0.69i59l3j69i57j69i60l4.651j0j4&sourceid=chrome&ie=UTF-8".format(x)
  results = []

  req = Request(link, headers = {"User-Agent": "Mozilla/5.0"})
  webpage = urlopen(req).read()
  with requests.Session() as c:
    soup = BeautifulSoup(webpage, "html.parser")
    #print(soup)
    for item in soup.find_all('div', attrs = {"class": "RWuggc"}):
      results.append(item)

  titles = []
  for word in results:
    for j in word:
      for i in j:
        for p in i:
          titles.append(p)

  end_of_titles = len(titles)
  movie_index = randint(0, end_of_titles )
  movie_index_end = movie_index + 1
  return(str(*titles[movie_index:movie_index_end]))


In [47]:
import json 
import numpy as nphello
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tkinter import *
from tkinter import ttk

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle
import time

with open("intents.json") as file:
    data = json.load(file)


def chatbox():

    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    def chatLog():
        text = textBox.get()
        inp = text

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])


        textBox.delete(0, 'end')
        chat.configure(state='normal')
        chat.insert('end', 'User: ' + text + '\n')
        for i in data['intents']:
            if i['tag'] == tag:
                chat.insert('end', 'Moviebot: ' + np.random.choice(i['responses']) + '\n')
        if tag == "genres":
            chat.insert('end', 'Moviebot: ' + "Got one for you! How about.. '" + webscraper(inp) + "'?" + '\n')
            genre = inp
        if tag == "another":
            chat.insert('end', 'Moviebot: ' + webscraper(genre) + '\n')
        chat.configure(state='disabled')


    root = Tk()
    chatBox = Scrollbar(root)
    chat = Text(root, wrap='word', state='disabled', width=50,
                yscrollcommand=chatBox.set)
    chatBox.configure(command=chat.yview)

    chat.grid(row=0, columnspan=2, sticky='ewns')
    chatBox.grid(row=0, column=2, sticky='ns')
    Label(root, text="Input: ").grid(row=1, column=0)

    textBox = Entry(root, bd=0, width=40, bg="lightblue")
    textBox.grid(row=1, column=1)

    Button(root, text="Send", command=chatLog).grid(row=2, columnspan=2)
    root.mainloop()

chatbox()


In [ ]:
titles = []
for word in results:
  for j in word:
    for i in j:
      for p in i:
        titles.append(p)

print(titles)
